In [102]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("StatsBase")
Pkg.add("StatsModels")
Pkg.add("Statistics")
Pkg.add("GLM")



   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`
   Resolvi

In [103]:
using DataFrames
using Statistics
using StatsBase
using CSV
using StatsModels
using GLM

# 1. Loading and processing the data

In [104]:
# 1. Load the data, either from the url or from the path provided

path = "C:/Users/User/Documents/GitHub/CausalAI-Course-1/"
data = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame) 
select!(data, Not("rownames"))

Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [105]:
# 2. As in Group Assignment 1, generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

types = Dict(:occ2 => String, :ind2 => String)

extra_flexible_formula = @formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2)
X_extra_flexible = modelmatrix(extra_flexible_formula, data);

In [106]:
# 2.1. Generate the array for the outcome variable Y and normalize it
y = data[:, "lwage"];
y_nom = (y .- mean(y)) ./ std(y) 

5150-element Vector{Float64}:
 -1.2402545501637507
  1.581415943901389
 -0.9952235701262617
 -0.5888284120346895
  0.6858352078834439
 -0.8916287370734166
 -0.02502721376252582
 -0.02502721376252582
 -0.8518460993196265
 -0.02502721376252582
 -0.20974552813450043
 -0.8490387289010607
 -0.8490387289010607
  ⋮
  0.018263935171735916
  0.14207082714988994
  1.4866251590094772
  0.25770849799681306
 -0.2591347787012701
  0.3661886075001642
 -0.6009613493971904
 -0.48781225590462607
  0.29461938662075393
  1.1902001226386991
  0.9199427728221882
 -0.20974552813450043

In [107]:
# 2.2. Generate the array for the predictors X (do not generate an intercept) and normalize its colums.}

x = select(data, Not([:wage, :lwage]))
x = Matrix(x)  # Convertir el data de "x" a una matriz
x = convert(Matrix{Float64}, x)

# Normalización de las predictoras
for col in 1:size(x, 2)
    x[:, col] = (x[:, col] .- mean(x[:, col])) ./ std(x[:, col])
end

In [108]:
# 3. Split between training and testing samples. The testing sample should be 10% of the total.
train_sample = rand(Float64, size(data)[1]) .< 0.9
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_extra_flexible_train, X_extra_flexible_test = X_extra_flexible[train_sample, :], X_extra_flexible[test_sample, :];

# 2. Creating the Lasso Cross-Validation Procedure

In [109]:
# 4. Program a function that generates a logarithmically spaced grid. The input arguments should be the lower and upper bounds of the grid, 
# as well as the natural logarithm of the spacing between each element of the grid. 
# The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, 
# they will be equally spaced. This will be the grid of values for λ values to try during cross-validation.

function generar_cuadrícula_logaritmica(limite_inferior, limite_superior, log_espaciado)
    # Número de elementos en la cuadrícula
    num_elementos = Int(ceil( (log(limite_superior) - log(limite_inferior)) / log_espaciado ) + 1)
    # Cuadrícula logarítmicamente espaciada
    cuadrícula = exp.(range(log(limite_inferior), log(limite_superior), length=num_elementos))

    return cuadrícula
end

generar_cuadrícula_logaritmica (generic function with 1 method)

In [111]:
# 5. Program a function to generate k folds. It should take as input the array to be split rowwise and the number of folds desired. 
# It should output a list of k 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, 
# and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. 
# You can aid yourself with third party packages like numpy in Python or Statistics in Julia, 
# but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

function generar_k_pliegues(arreglo, k)
    num_filas = size(arreglo, 1)
    tamaño_pliegue = div(num_filas, k) 
    resto = num_filas % k

    # Lista de pliegues 
    pliegues = [BitVector(falses(num_filas)) for _ in 1:k]

    # Asignación de filas a los pliegues
    inicio = 1
    for i in 1:k
        fin = inicio + tamaño_pliegue - 1 + (i <= resto) 
        pliegues[i][inicio:fin] .= true
        inicio = fin + 1
    end

    # Verificador que todos los pliegues 
    @assert reduce(|, vcat(map(x -> convert(Array{Bool}, x), pliegues)...)) == trues(num_filas) "Los pliegues no cubren todas las filas" 

    return pliegues
end


generar_k_pliegues (generic function with 3 methods)

In [112]:
using Pkg
Pkg.add("GLMNet")

using GLMNet

   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`


In [113]:
# 7. Program a function that integrates those that you programmed in the last two items to find the value of λ 
# that minimizes the testing mean square error across folds. It should take the following inputs:

# Y: an array for the outcome variable.
# X: an array of predictors.
# lambda_bounds: the lower and upper bounds of the grid of lambda values.
# k: number of folds

#The output should be a dictionary (a list in R) with the following entries:

# optimal_lambda: The lambda that minimizes the testing MSE across folds.
# optimal_coef: An array with the coefficients found for the optimal lambda
# all_lambdas: The grid of lambdas.
# all_mse: An array with the testing MSE across folds for each lambda.

# The procedure goes as follows:

# With each lambda in the grid For each split Train and test a lasso model. With this, 
# you can get the testing MSE associated with the value of lambda in each iteration. 
#Feel free to use a third party Lasso estimator, as long as it is not one that calculates the optimal lambda value. 
#Instead, the third party estimator you use should only estimate the regression with the penalty weight provided so you can get the testing MSE with that specific weight.

function encontrar_lambda_optimo(y, X, lambda_bounds, k)
    limite_inferior_lambda, limite_superior_lambda = lambda_bounds
    log_espaciado_lambda = log(2) 

    # Cuadrícula de valores lambda
    lambda_valores = generar_cuadrícula_logaritmica(limite_inferior_lambda, limite_superior_lambda, log_espaciado_lambda)

    # Los k pliegues
    pliegues = generar_k_pliegues(X, k)

    # Los errores medios para cada valor de lambda
    errores_medios = zeros(length(lambda_valores))

    # Validación cruzada
    for (i, lambda) in enumerate(lambda_valores)
        error_total = 0.0
        for j in 1:k
            # Datos Train y Test para este pliegue
            X_train = X[.!pliegues[j], :]
            y_train = y[.!pliegues[j]]
            X_test = X[pliegues[j], :]
            y_test = y[pliegues[j]]

            # Ajustar el modelo Lasso
            modelo_lasso = glmnet(X_train, y_test, alpha=1, lambda=lambda) 

            # Predecir en los datos de prueba
            y_prediccion = predict(modelo_lasso, X_test)

            # Error cuadrático medio para este pliegue
            error_cuadratico_medio = mean((y_prueba .- y_prediccion).^2)
            error_total += error_cuadratico_medio
        end

        # Error medio para este valor de lambda
        errores_medios[i] = error_total / k 
    end

    # Indice del lambda con el menor error medio
    indice_optimo = argmin(errores_medios)

    # Lambda óptimo y sus coeficientes 
    lambda_optimo = lambda_valores[indice_optimo]
    coeficientes_optimos = coef(glmnet(X, y, alpha=1, lambda=lambda_optimo))

    # Resultados en un diccionario
    return Dict(
        "optimal_lambda" => lambda_optimo,
        "optimal_coef" => coeficientes_optimos,
        "all_lambdas" => lambda_valores,
        "all_mse" => errores_medios
    )
end

encontrar_lambda_optimo (generic function with 2 methods)

In [114]:
#Answer 8
using GLM, CSV, DataFrames

# Load the data from the CSV file
url = "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"
data = CSV.read(download(url), DataFrame)

# Select the formula for the extra flexible model, adjust according to the model you want
ols_formula = @formula(lwage ~ sex + exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + occ2 + ind2)

# Split the data into training and testing samples (10% for testing)
train_sample = rand(Float64, size(data)[1]) .< 0.9
test_sample = .!(train_sample)

# Fit the OLS model using the training sample
ols_model = lm(ols_formula, data[train_sample, :])

# Display the OLS model results
println(coef(ols_model))    # Coefficients
println(stderror(ols_model))  # Standard Errors
println(r2(ols_model))        # R-squared value


[2.8472460459236726, -0.10308437508818606, 0.03173352132043804, -0.12871864358074683, 0.03328319060102533, -0.00400097464555948, 0.11074886646340922, 0.2348180577578236, 0.49802948672446795, 0.695314390870509, 0.0024155354487575866, 0.030729846206997605, 0.020736383665778433, -0.01977576889248053, -0.013306983228836023]
[0.06535709990195009, 0.015259315922103828, 0.01079410183985792, 0.1096661403189388, 0.04122402970825441, 0.005087690738598404, 0.04953877848157821, 0.04965483809464179, 0.05057955827169425, 0.05338564982071242, 0.019704326301675307, 0.021336404683305794, 0.021044604413239288, 0.0012191667607699523, 0.0013743100050750473]
0.239152630293616


In [115]:
#Answer 9

function encontrar_lambda_optimo(y, X, lambda_bounds, k)
    limite_inferior_lambda, limite_superior_lambda = lambda_bounds
    log_espaciado_lambda = 0.1  # Espaciado en logaritmo natural


    n_lambda = ceil(Int, (log(limite_superior_lambda) - log(limite_inferior_lambda)) / log_espaciado_lambda) + 1
    lambda_valores = exp.(range(log(limite_inferior_lambda), log(limite_superior_lambda), length=n_lambda))

    pliegues = generar_k_pliegues(X, k)
    errores_medios = zeros(length(lambda_valores))

    for (i, lambda) in enumerate(lambda_valores)
        error_total = 0.0
        for j in 1:k
            X_train_fold = X[.!pliegues[j], :]
            y_train_fold = y[.!pliegues[j]]
            X_test_fold = X[pliegues[j], :]
            y_test_fold = y[pliegues[j]]

            modelo_lasso = glmnet(X_train_fold, y_train_fold, alpha=1, lambda=lambda)
            y_prediccion = predict(modelo_lasso, X_test_fold)

            error_cuadratico_medio = mean((y_test_fold .- y_prediccion).^2)
            error_total += error_cuadratico_medio
        end

        errores_medios[i] = error_total / k
    end

    indice_optimo = argmin(errores_medios)
    lambda_optimo = lambda_valores[indice_optimo]
    coeficientes_optimos = coef(glmnet(X, y, alpha=1, lambda=lambda_optimo))

    return Dict(
        "optimal_lambda" => lambda_optimo,
        "optimal_coef" => coeficientes_optimos,
        "all_lambdas" => lambda_valores,
        "all_mse" => errores_medios
    )
end


encontrar_lambda_optimo (generic function with 2 methods)

In [116]:

using Pkg
Pkg.add("HDMjl")  # Solo si no lo has instalado ya
using HDMjl

   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`


In [117]:
function encontrar_lambda_optimo(y::Vector{Float64}, X::Matrix{Float64}, lambda_bounds::Tuple{Float64, Float64}, k::Int64)
    pliegues = generar_k_pliegues(X, k)
    

    @assert length(unique(vcat(pliegues...))) == size(X, 1) "Los pliegues no cubren todas las filas"
    

end


encontrar_lambda_optimo (generic function with 2 methods)

In [118]:
function generar_cuadrícula_logarítmica(limite_inferior_log, limite_superior_log, espaciado_log)

    log_grid = limite_inferior_log:espaciado_log:limite_superior_log
    
    lambda_grid = exp.(log_grid)
    return lambda_grid
end


limite_inferior_log = -7
limite_superior_log = 7
espaciado_log = 0.1

lambda_grid = generar_cuadrícula_logarítmica(limite_inferior_log, limite_superior_log, espaciado_log)


141-element Vector{Float64}:
    0.0009118819655545162
    0.0010077854290485105
    0.0011137751478448032
    0.001230911902673481
    0.0013603680375478939
    0.0015034391929775724
    0.001661557273173934
    0.0018363047770289071
    0.002029430636295734
    0.0022428677194858034
    0.0024787521766663585
    0.0027394448187683684
    0.0030275547453758153
    ⋮
  365.0374678653289
  403.4287934927351
  445.85777008251677
  492.7490410932563
  544.571910125929
  601.8450378720822
  665.1416330443618
  735.0951892419727
  812.4058251675433
  897.8472916504176
  992.2747156050262
 1096.6331584284585

In [119]:
function dividir_k_pliegues(X, y, k)
    n = size(X, 1)
    indices = shuffle(1:n)
    tamaño_pliegue = div(n, k)
    pliegues = []

    for i in 1:k
        if i < k
            pliegue_indices = indices[(i-1)*tamaño_pliegue+1:i*tamaño_pliegue]
        else
            pliegue_indices = indices[(i-1)*tamaño_pliegue+1:end]
        end
        push!(pliegues, pliegue_indices)
    end
    return pliegues
end


dividir_k_pliegues (generic function with 1 method)

In [120]:
# Define the bounds and spacing of the natural logarithm
log_lambda_min = -7
log_lambda_max = 7
spacing = 0.1

# Create the grid of natural logarithms
log_lambda_grid = log_lambda_min:spacing:log_lambda_max

# Apply the exponential function to get the grid of lambda values
lambda_grid = exp.(log_lambda_grid)

# Display the result
lambda_grid


141-element Vector{Float64}:
    0.0009118819655545162
    0.0010077854290485105
    0.0011137751478448032
    0.001230911902673481
    0.0013603680375478939
    0.0015034391929775724
    0.001661557273173934
    0.0018363047770289071
    0.002029430636295734
    0.0022428677194858034
    0.0024787521766663585
    0.0027394448187683684
    0.0030275547453758153
    ⋮
  365.0374678653289
  403.4287934927351
  445.85777008251677
  492.7490410932563
  544.571910125929
  601.8450378720822
  665.1416330443618
  735.0951892419727
  812.4058251675433
  897.8472916504176
  992.2747156050262
 1096.6331584284585

In [121]:
#Answer 10
using Pkg
Pkg.add("HDMjl")


   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`


In [122]:
using HDMjl, DataFrames, CSV

# Cargar los datos
data = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame) 
##data = CSV.File("https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv") |> DataFrame

# Definir la variable dependiente y las independientes
y = data.wage
X = select(data, Not(:wage))


Row,rownames,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,30,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,43,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,44,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,47,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [128]:
using Pkg
Pkg.add("HDMjl")  
using HDMjl
using DataFrames, CSV
using Random
using StatsBase 

   Resolving package versions...
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Project.toml`
  No Changes to `C:\Users\Maricielo\Pictures\git vs code\CausalAI-Course\Labs\Assignment\Assignment_2\Group4_Assignment2\MyNewProject\Manifest.toml`


In [ ]:
# Load the data
data = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame) 

# Define the dependent and independent variables
y = data.wage
X = select(data, Not(:wage))

# Set seed for reproducibility
Random.seed!(123)

# Split the data into training and testing sets
train_indices = sample(1:nrow(data), round(Int, 0.8 * nrow(data)), replace=false)
X_train = X[train_indices, :]
y_train = y[train_indices]
X_test = X[setdiff(1:nrow(data), train_indices), :]
y_test = y[setdiff(1:nrow(data), train_indices)]

# Define the hdm_lasso function
function hdm_lasso(X, y, λ)
    model = HDMjl.lasso(X, y, λ=λ)
    return model
end

lambda_optimo = 0.1 

modelo_lasso = hdm_lasso(X_train, y_train, lambda_optimo)

# Display the results
println(lasso_model)


In [170]:
using Random

Random.seed!(123)

train_indices = sample(1:nrow(data), round(Int, 0.8 * nrow(data)), replace=false)

4120-element Vector{Int64}:
 2685
 3023
 4589
  986
 2710
 2015
  237
 4808
 2994
 1692
 2719
 4309
  223
    ⋮
 3947
 2781
 4390
 2580
 2243
  371
 4298
 1617
  460
 3053
 2308
 2812

In [159]:
#Answer 11
using CSV
using DataFrames

# Cargar los datos
data = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame) 

# Definir la variable dependiente y las independientes
y = data.wage
X = select(data, Not(:wage))
using CSV
using DataFrames
using GLM
using Random
using HDMjl  # Make sure to have the HDM package installed
using StatsBase  # For calculating MSE and R^2

# Load the data
data = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame)

# Define the dependent and independent variables
y = data.wage
X = select(data, Not(:wage))

# Set seed for reproducibility
Random.seed!(123)

# Split the data into training and testing sets
train_indices = sample(1:nrow(data), round(Int, 0.8 * nrow(data)), replace=false)
y_train = y[train_indices]
X_train = X[train_indices, :]
y_test = y[setdiff(1:nrow(data), train_indices)]
X_test = X[setdiff(1:nrow(data), train_indices), :]

# Fit the OLS model
ols_model = glm((wage ~ .), data[train_indices, :], Normal(), IdentityLink())
ols_predictions = predict(ols_model, data[setdiff(1:nrow(data), train_indices), :])

# Calculate MSE and R^2 for OLS model
ols_mse = mean((ols_predictions - y_test).^2)
ols_r2 = 1 - (sum((y_test .- ols_predictions).^2) / sum((y_test .- mean(y_test)).^2))

println("OLS Model - MSE: ", ols_mse, ", R^2: ", ols_r2)

# Fit the Lasso model using cross-validation

lambda_cv = 0.1 
lasso_cv_model = lasso(X_train, y_train, alpha=1, lambda=lambda_cv)
lasso_cv_predictions = predict(lasso_cv_model, X_test)

# Calculate MSE and R^2 for Lasso CV model
lasso_cv_mse = mean((lasso_cv_predictions - y_test).^2)
lasso_cv_r2 = 1 - (sum((y_test .- lasso_cv_predictions).^2) / sum((y_test .- mean(y_test)).^2))

println("Lasso CV Model - MSE: ", lasso_cv_mse, ", R^2: ", lasso_cv_r2)

# Fit the Lasso model using theoretical lambda
lambda_optimo = 0.1  
lasso_optimal_model = hdm_lasso(X_train, y_train, lambda=lambda_optimo)
lasso_optimal_predictions = predict(lasso_optimal_model, X_test)

# Calculate MSE and R^2 for Lasso theoretical lambda model
lasso_optimal_mse = mean((lasso_optimal_predictions - y_test).^2)
lasso_optimal_r2 = 1 - (sum((y_test .- lasso_optimal_predictions).^2) / sum((y_test .- mean(y_test)).^2))

println("Lasso Theoretical Lambda Model - MSE: ", lasso_optimal_mse, ", R^2: ", lasso_optimal_r2)



Row,rownames,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,15,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,18,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,19,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,30,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,43,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,44,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,47,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


In [162]:
println(first(data, 5))


5×21 DataFrame
 Row │ rownames  wage      lwage    sex      shs      hsg      scl      clg      ad       mw       so       we       ne       exp1     exp2     exp3     exp4     occ      occ2   ind      ind2  
     │ Int64     Float64   Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Int64  Float64  Int64 
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │       10   9.61538  2.26336      1.0      0.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0      1.0      7.0     0.49    0.343   0.2401   3600.0     11   8370.0     18
   2 │       12  48.0769   3.8728       0.0      0.0      0.0      0.0      1.0      0.0      0.0      0.0      0.0      1.0     31.0     9.61   29.791  92.3521   3050.0     10   5070.0      9
   3 │       15  